In [1]:
def read_data(filename):
    with open(filename, 'r', -1, 'utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data

train = read_data('navershopping_all.txt')

lines = open('stopwords.txt','r',-1,'utf-8')
stopwords=[word.replace('\n','') for word in lines.readlines()]
lines = open('stoptags.txt','r',-1,'utf-8')
stoptags=[word.replace('\n','') for word in lines.readlines()]

In [2]:
train_len = len(train)
train_text = []
train_y = []

for i in range(0, train_len):
    train_text.insert(i, train[i][1])
for i in range(0, train_len):
    train_y.insert(i, train[i][0])

In [3]:
print(len(train))
print(train_text[0:10])
print(train_y[0:10])

51815
['타사 제품 쓰다 처음으로 바꿔봤는데 잘못된 선택이었네요 얇은만큼 기저귀 금방 빵빵해지네요 다른말로 흡수력이 좋진 않은거겠죠 두시간만에 엄청 빵빵해지고 내의까지 습해지는게 기저귀 터진줄 알았습니다 거기다 기저귀 갈면서보니 찝찝하게 충전제인지 투명한 알갱이가 애기 몸에 붙어있네요 저렴한 맛에 구매했다가 돈버린듯.', '5.5키로 우리애기...8키로쯤된줄알았어요 너무작네요 다못쓸까봐 쉬한번했는데도 그냥 막갈아주고있어요 돈아까워요 ㅠㅠ', '아이가 핑크퐁을 좋아해서 보자마자 좋아하긴하는데.. 흡수력이 별론지..밤에 두번이나 샜네요', '스티커에 혹해서 구입했는데 오지않았구요. 상품은 팬티구입했는데 그냥그래요. 쓸만합니다 재구매는..글쎄요~', '아기 엉덩이에 자국이 남고요 거친 느낌이 있어요 싸게 산건 맞네요', '다시는 구매안할듯요 인형은 아기가 좋아는하네요', '이거 괜찮다 한사람 ㄴㅇ', '싼맛에 사보았는데.....ㅠㅠ', '배송 총알이네요 마미포코 쿨웨이브 쓰고 있는데 비슷해요 6팩은 6월 제조일이고 인형,파우치와 같이 있던 1팩은 3월 제조일이네요 이렇게 대량 구성 구매는 처음인데 핑크퐁 인형에 혹해서 샀어요 한동안 기저귀 걱정 없겠네요', '아기가 핑크퐁을 워낙 좋아해서 샀는데 인형은 역시 엄청 좋아하네요. 기저귀는 하기스 썸머보다는 길이가 짧아요. 살짝 두껍고- 마미포코보다는 뜯기 좀 수월한것 같아요. 싸게 잘 샀어요~ 안새면 재구매할께요~']
['2', '2', '2', '2', '2', '2', '2', '2', '5', '5']


In [4]:
from konlpy.tag import Twitter
twit = Twitter()

def tokenize(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1] not in stoptags]
    # norm, stem은 optional
    return result

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(tokenizer=tokenize, min_df)

In [ ]:
count_train = count_vect.fit_transform(train_text)

In [8]:
count_train.shape

(51815, 12413)

In [9]:
print(count_vect.vocabulary_)

{'타다/Verb': 10890, '제품/Noun': 9491, '쓰다/Verb': 6741, '처음/Noun': 10293, '으로/Josa': 8335, '바꾸다/Verb': 4189, '보다/Verb': 4708, '잘못/Noun': 9021, '되다/Verb': 2626, '선택/Noun': 5806, '이다/Verb': 8467, '얇다/Adjective': 7155, '기저귀/Noun': 1257, '금방/Noun': 1150, '빵빵/Noun': 5244, '해/Josa': 11875, '지네/Noun': 9846, '요/Josa': 7974, '다른/Noun': 2063, '말로/Noun': 3529, '흡수/Noun': 12366, '력/Suffix': 3158, '이/Josa': 8433, '좋다/Adjective': 9617, '않다/Verb': 7021, '두/Determiner': 2637, '시간/Noun': 6404, '만에/Josa': 3474, '엄청/Adverb': 7355, '지고/Noun': 9827, '내의/Noun': 1701, '까지/Josa': 1318, '습/Noun': 6386, '지다/Verb': 9851, '터지다/Verb': 10971, '알다/Verb': 7026, '거기/Noun': 333, '다/Josa': 2040, '갈다/Verb': 175, '찌다/Verb': 10143, '충전/Noun': 10516, '제인/Noun': 9475, '지/Josa': 9822, '투명하다/Adjective': 11063, '알갱이/Noun': 7024, '가/Josa': 0, '애기/Noun': 7093, '몸/Noun': 3852, '에/Josa': 7395, '붙다/Verb': 5044, '저렴/Noun': 9196, '한/Josa': 11771, '맛/Noun': 3553, '구매/Noun': 887, '하다/Verb': 11688, '돈/Noun': 2548, '버리다/Verb': 4519, '키로/Suff

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(count_train)
tf_train = tf_transformer.transform(count_train)
print(tf_train.shape)

(51815, 12413)


In [11]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000)
svd_tf_train = svd.fit_transform(tf_train)

In [13]:
print(svd.explained_variance_ratio_.sum())

0.867607289423


In [15]:
import numpy as np
svd_tf_train2 = svd_tf_train.astype(np.float)
train_y2 = np.array(train_y).astype(np.float)
print(svd_tf_train2)
print(train_y2)

[[  3.06195663e-01   6.15778440e-02   1.09068641e-02 ...,  -5.85217506e-03
    7.24880338e-03   4.41717489e-03]
 [  2.26788617e-01  -7.93694137e-02  -6.47888504e-02 ...,  -3.21684056e-03
   -3.84500397e-03  -5.73878555e-04]
 [  1.90331772e-01   1.41167568e-01  -1.32742699e-01 ...,   3.70709003e-03
    6.17443315e-03  -1.33668849e-03]
 ..., 
 [  3.78702356e-02   2.97458241e-02  -7.12347124e-03 ...,  -7.89079506e-03
   -3.71497718e-04  -1.91496883e-02]
 [  2.10896784e-01   1.40910212e-01  -1.00055767e-01 ...,   1.22334634e-03
   -1.14315180e-02  -2.90195789e-03]
 [  3.13428364e-01   2.48807672e-01   3.74737527e-01 ...,   6.46376629e-03
    5.15094086e-04  -1.34440393e-03]]
[ 2.  2.  2. ...,  1.  5.  5.]


In [16]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression(normalize=True)
reg_model = reg.fit(svd_tf_train2, train_y2)

In [18]:
print(len(reg.coef_))
print(reg.intercept_)
print(max(reg.coef_))

1000
1.77519706499
4.60670288592


In [22]:
print(reg_model.score(svd_tf_train2, train_y2))
for i in range(0,10):
    print(train_text[i], reg_model.predict(svd_tf_train2[i]))

0.614655776299
타사 제품 쓰다 처음으로 바꿔봤는데 잘못된 선택이었네요 얇은만큼 기저귀 금방 빵빵해지네요 다른말로 흡수력이 좋진 않은거겠죠 두시간만에 엄청 빵빵해지고 내의까지 습해지는게 기저귀 터진줄 알았습니다 거기다 기저귀 갈면서보니 찝찝하게 충전제인지 투명한 알갱이가 애기 몸에 붙어있네요 저렴한 맛에 구매했다가 돈버린듯. [ 3.20656597]
5.5키로 우리애기...8키로쯤된줄알았어요 너무작네요 다못쓸까봐 쉬한번했는데도 그냥 막갈아주고있어요 돈아까워요 ㅠㅠ [ 2.30886414]
아이가 핑크퐁을 좋아해서 보자마자 좋아하긴하는데.. 흡수력이 별론지..밤에 두번이나 샜네요 [ 2.93965336]
스티커에 혹해서 구입했는데 오지않았구요. 상품은 팬티구입했는데 그냥그래요. 쓸만합니다 재구매는..글쎄요~ [ 2.97879557]
아기 엉덩이에 자국이 남고요 거친 느낌이 있어요 싸게 산건 맞네요 [ 2.32308202]
다시는 구매안할듯요 인형은 아기가 좋아는하네요 [ 2.55216524]
이거 괜찮다 한사람 ㄴㅇ [ 2.28608229]
싼맛에 사보았는데.....ㅠㅠ [ 2.96378139]
배송 총알이네요 마미포코 쿨웨이브 쓰고 있는데 비슷해요 6팩은 6월 제조일이고 인형,파우치와 같이 있던 1팩은 3월 제조일이네요 이렇게 대량 구성 구매는 처음인데 핑크퐁 인형에 혹해서 샀어요 한동안 기저귀 걱정 없겠네요 [ 3.81788552]
아기가 핑크퐁을 워낙 좋아해서 샀는데 인형은 역시 엄청 좋아하네요. 기저귀는 하기스 썸머보다는 길이가 짧아요. 살짝 두껍고- 마미포코보다는 뜯기 좀 수월한것 같아요. 싸게 잘 샀어요~ 안새면 재구매할께요~ [ 4.62798493]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarn

In [ ]:
import tensorflow as tf

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer().fit(count_train)
tf_train = tf_transformer.transform(count_train)

print(tf_train.shape)

(51815, 1011)


In [ ]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg.fit([[0, 0], [1, 1]], [0, 1])
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
reg.predict([[1, 1]])

In [227]:
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB()
clf_NB_model = clf_NB.fit(tf_train, train_y)

In [228]:
print(clf_NB_model.score(tf_train, train_y), clf_NB_model.score(tf_baemin, baemin_y), clf_NB_model.score(tf_fan, fan_y))

0.957320499574 0.925 0.849246231156


In [229]:
from sklearn.linear_model import LogisticRegression
clf_lgt = LogisticRegression(C=10)
clf_lgt_model = clf_lgt.fit(tf_train, train_y)

In [230]:
print(clf_lgt_model.score(tf_train, train_y), clf_lgt_model.score(tf_baemin, baemin_y), clf_lgt_model.score(tf_fan, fan_y))

0.987188899161 0.915 0.834170854271


In [231]:
kang = read_data('kang.txt')

kang_len = len(kang)
kang_text =[]
for i in range(0, kang_len):
    kang_text.insert(i, kang[i][0])

In [232]:
count_kang = count_vect.transform(kang_text)
tf_kang = tf_transformer.transform(count_kang)

In [233]:
label = clf_lgt_model.predict(tf_kang)
prob = clf_lgt_model.predict_proba(tf_kang)
data = []
for i in range(0, len(kang_text)-1):
    data.append([label[i], prob[i], kang_text[i]])

In [234]:
conf = []
for i in range(0, len(kang_text)-1):
    if abs(data[i][1][0] - data[i][1][1]) > 0.4:
        conf.append([label[i], prob[i], kang_text[i]])

In [235]:
pos = []
for i in range(0,len(conf)-1):
    if conf[i][0]=='1':
        pos.append(conf[i][2])

neg = []
for i in range(0,len(conf)-1):
    if conf[i][0]=='0':
        neg.append(conf[i][2])

In [236]:
pos_doc = ''.join(pos)
neg_doc = ''.join(neg)
both = [pos_doc, neg_doc]

In [237]:
from konlpy.tag import Twitter
twit = Twitter()

def tokenize(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1] not in stoptags_tfidf]
    # norm, stem은 optional
    return result

In [238]:
count_vect = CountVectorizer(tokenizer=tokenize)
count_both = count_vect.fit_transform(both)
count_pos = count_vect.transform(pos)
count_neg = count_vect.transform(neg)

tf_transformer = TfidfTransformer().fit(count_both)
tf_both = tf_transformer.transform(count_both)
tf_pos = tf_transformer.transform(count_pos)
tf_neg = tf_transformer.transform(count_neg)

In [239]:
tf_both_array = tf_both.toarray()
sort_index = tf_both_array.argsort(axis=1)

In [240]:
count_dict = count_vect.vocabulary_
for index in sort_index[0][-20:]:
    important_word = index
    key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
    print(key)

여성/Noun
청문회/Noun
사람/Noun
다음/Noun
한국/Noun
야당/Noun
지지/Noun
뉴스/Noun
문재인/Noun
김상조/Noun
후보/Noun
것/Noun
외교/Noun
대통령/Noun
외교부/Noun
후보자/Noun
임명/Noun
국민/Noun
장관/Noun
강경화/Noun


In [241]:
count_dict = count_vect.vocabulary_
for index in sort_index[1][-20:]:
    important_word = index
    key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
    print(key)

문제/Noun
네이버/Noun
왜/Noun
청문회/Noun
의혹/Noun
국민/Noun
다음/Noun
안/Noun
대통령/Noun
후보/Noun
국민의당/Noun
야당/Noun
후보자/Noun
김상조/Noun
것/Noun
장관/Noun
반대/Noun
뉴스/Noun
임명/Noun
강경화/Noun


In [254]:
from sklearn.decomposition import LatentDirichletAllocation

for i in range(2,6):
    lda = LatentDirichletAllocation(n_topics=i, learning_method="batch", max_iter=10, random_state=0)
    document_topics = lda.fit_transform(tf_pos)
    print(i, lda.perplexity(tf_pos))

# Recommend you to use the number of topics which has minimum value of perplexity

2 3496.62726509
3 6973.83975527
4 13843.5320778


In [261]:
lda = LatentDirichletAllocation(n_topics=4, learning_method="batch", max_iter=10, random_state=0)
document_topics = lda.fit_transform(tf_pos)
print("lda.components_.shape: {}".format(lda.components_.shape))

lda.components_.shape: (4, 16653)


In [263]:
import numpy as np

sorting = np.argsort(lda.components_, axis=1)[:,::-1]
feature_names = np.array(count_vect.get_feature_names())

import mglearn
mglearn.tools.print_topics(topics=range(4), feature_names=feature_names, sorting=sorting, 
                           topics_per_chunk=5, n_words=20)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
발언/Noun       강경화/Noun      강경화/Noun      년전/Noun       
동영상/Noun      장관/Noun       김상조/Noun      주목/Noun       
최고/Noun       임명/Noun       유시민/Noun      년/Noun        
표시/Noun       후보자/Noun      모두/Noun       변경/Noun       
영어/Noun       국민/Noun       후보/Noun       청문회/Noun      
마무리/Noun      외교부/Noun      우리/Noun       아침/Noun       
다큐/Noun       대통령/Noun      뭉쳐/Noun       최경환/Noun      
와우/Noun       외교/Noun       링크/Noun       안/VerbPrefix  
언니/Noun       후보/Noun       근절/Noun       위증죄/Noun      
클라스/Noun      것/Noun        장관/Noun       형질/Noun       
동시/Noun       지지/Noun       구/Noun        김영희/Noun      
재직/Noun       뉴스/Noun       한국/Noun       역시/Noun       
조명/Noun       문재인/Noun      연설/Noun       대지/Noun       
사표/Noun       김상조/Noun      사람/Noun       햇발/Noun       
혹평/Noun       다음/Noun       시정/Noun       인간/Noun       
부양/Noun       야당/Noun       위장전

In [259]:
lda = LatentDirichletAllocation(n_topics=3, learning_method="batch", max_iter=10, random_state=0)
document_topics = lda.fit_transform(tf_neg)
print("lda.components_.shape: {}".format(lda.components_.shape))

In [260]:
import numpy as np

sorting = np.argsort(lda.components_,axis=1)[:,::-1]
feature_names = np.array(count_vect.get_feature_names())

import mglearn
mglearn.tools.print_topics(topics=range(3),feature_names=feature_names,sorting=sorting, 
                           topics_per_chunk=5,n_words=20)

topic 0       topic 1       topic 2       
--------      --------      --------      
강경화/Noun      강경화/Noun      강경화/Noun      
뉴스/Noun       임명/Noun       의혹/Noun       
국제/Noun       반대/Noun       뉴스/Noun       
다음/Noun       뉴스/Noun       부동산/Noun      
성명/Noun       장관/Noun       김상조/Noun      
피해자/Noun      것/Noun        보도/Noun       
지지/Noun       국민의당/Noun     위장전입/Noun     
홈페이지/Noun     야당/Noun       손석희/Noun      
위안부/Noun      후보자/Noun      왜곡/Noun       
긴급/Noun       김상조/Noun      기획/Noun       
내/Noun        후보/Noun       과장/Noun       
기자회견/Noun     대통령/Noun      추측/Noun       
답변/Noun       안/Noun        상당수/Noun      
장관/Noun       국민/Noun       다음/Noun       
부모/Noun       왜/Noun        뉴스룸/Noun      
아줌마/Noun      못/VerbPrefix  사과/Noun       
친정/Noun       강행/Noun       논란/Noun       
직원/Noun       다음/Noun       청문회/Noun      
전여옥/Noun      김이수/Noun      후보자/Noun      
기구/Noun       외교부/Noun      네이버/Noun      


